
# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS209A Introduction to Data Science

## Stock Data: Inter-Day for American And Chinese exchanges

**Harvard University**<br/>
**Fall 2019**<br/>
**Instructors**: Pavlos Protopapas, Kevin Rader, and Chris Tanner

**Students**: Aaron Jacobson, Hayden Joy, Priya Veerarghavan, Corin Wagen
<hr style="height:2pt">



In [1]:
#RUN THIS CELL 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)


### INSTRUCTIONS

- Below we read in various stock exchange csv data. The main split currently is an AMEX/NASDAQ list, and the SZSE exchange. But American stocks can be listed on the Chinese exchange and Chinese stocks can be listed on the American exchange so we need a better way to correlated the stocks with the respective countries.
- One thing that is worth investigation are stocks specifically targeted by the trade war on either side.
- Note that one basic way to measure volatility is the change in daily volume.

In [17]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind
pd.set_option('display.max_columns', 25)

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LassoCV

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import metrics

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import statsmodels.api as sm
from statsmodels.api import OLS

import seaborn as sns
sns.set()
import warnings
warnings.filterwarnings('ignore')

American Stock Exchanges:

In [18]:
#stocks = pd.read_csv("MyData.csv")
stocks = pd.read_csv("nasdaq_amex.csv") #AMEX, NASDAQ
display(stocks.head())
def get_stock(ticker):
    return(stocks.iloc[:,:][stocks["ticker"] == ticker])
get_stock("AAPL")
len(stocks.index)

,Unnamed: 0,price.open,price.high,price.low,price.close,volume,price.adjusted,ref.date,ticker
0,12,40.509998,40.849998,39.570000,39.799999,9028700.0,38.629791,2016-11-01,AAL
1,13,39.720001,40.250000,39.619999,39.709999,5274500.0,38.542442,2016-11-02,AAL
2,14,39.779999,40.240002,39.299999,39.380001,6006500.0,38.318638,2016-11-03,AAL
3,15,39.270000,40.220001,39.240002,39.740002,6717500.0,38.668938,2016-11-04,AAL
4,16,40.389999,41.650002,40.310001,41.490002,8334200.0,40.371773,2016-11-07,AAL


653999

Chinese Stock Exchange:

In [20]:
# Chinese exchanges for now is this exchange: "SZSE"
stocks_SZSE = pd.read_csv("szse_exchange1.csv") #"SZSE"
display(stocks.head())
def get_stock(ticker):
    return(stocks.iloc[:,:][stocks["ticker"] == ticker])
len(stocks_SZSE.index)


,Unnamed: 0,price.open,price.high,price.low,price.close,volume,price.adjusted,ref.date,ticker
0,12,40.509998,40.849998,39.570000,39.799999,9028700.0,38.629791,2016-11-01,AAL
1,13,39.720001,40.250000,39.619999,39.709999,5274500.0,38.542442,2016-11-02,AAL
2,14,39.779999,40.240002,39.299999,39.380001,6006500.0,38.318638,2016-11-03,AAL
3,15,39.270000,40.220001,39.240002,39.740002,6717500.0,38.668938,2016-11-04,AAL
4,16,40.389999,41.650002,40.310001,41.490002,8334200.0,40.371773,2016-11-07,AAL


1907049

---